In [26]:
import  pickle
import time
from typing import Callable, Dict, Tuple

import cv2
import numpy as np
import pyautogui
from sklearn.ensemble import RandomForestClassifier

from kotoamatsukami import HandDetector



supported_windows = ["edge", "powerpoint"]


def load_model() -> RandomForestClassifier:
    with open("models/decision_tree", "rb") as file:
        model = pickle.load(file)

    return model


def get_gesture_labes() -> Dict[int, str]:
    labels = {
        0: "click",
        1: "closed",
        2: "down",
        3: "mouse_tracking",
        4: "negative_closed",
        5: "negative_mouse_tracking",
        6: "negative_side",
        7: "negative_up",
        8: "side",
        9: "up"
    }

    return labels


def mouse_tracking_command(x_offset: int, y_offset: int) -> None:
    pyautogui.moveRel(xOffset=x_offset, yOffset=y_offset)
    time.sleep(0.1)


def closed_command(hand_type: str) -> None:
    if hand_type == "Right":
        pyautogui.hotkey("playpause")
        time.sleep(0.1)
        time.sleep(3)


def down_command(hand_type: str) -> None:
    if hand_type == "Right":
        pyautogui.click()
        time.sleep(0.25)
        pyautogui.hotkey("pgdn")
        time.sleep(2)


def up_command(hand_type: str) -> None:
    if hand_type == "Right":
        pyautogui.click()
        time.sleep(0.25)
        pyautogui.hotkey("pgup")
        time.sleep(2)


def side_command(hand_type: str) -> None:
    # pyautogui.click()
    # time.sleep(0.25)
    active_window_full_name = pyautogui.getActiveWindowTitle()
    names = active_window_full_name.lower().split()

    if "edge" in names:
        if hand_type == "Right":
            pyautogui.hotkey("browserforward")

        else:
            pyautogui.hotkey("browserback")

    if "powerpoint" in names:
        if hand_type == "Right":
            pyautogui.hotkey("down")

        else:
            pyautogui.hotkey("up")

    time.sleep(2)


def null_command(hand_type: str) -> None:
    pass


def get_commands_table() -> Dict[int, Callable]:
    commands_table = {
        1: closed_command,
        2: up_command,
        8: side_command,
        9: down_command
    }

    return commands_table


def get_mouse_move_offset(hand: dict, previous_x: int, previous_y: int) -> Tuple[float, float]:
    current_x = hand["pos_x"]
    current_y = hand["pos_y"]

    x_coef = -1920/1280         # Ta negativo para espelhar a imagem
    y_coef = 1080/720

    x_offset = (current_x - previous_x)*x_coef
    y_offset = (current_y - previous_y)*y_coef

    return x_offset, y_offset


def main() -> None:
    print("Starting!!!")

    min_probability = 0.9
    commands_table  = get_commands_table()

    stream = cv2.VideoCapture(0)
    hand_detector = HandDetector(max_hands=1, detection_con=0.9, normalize=True)

    labels = get_gesture_labes()
    model  = load_model()

    i = 0

    previous_x = None
    previous_y = None

    was_clicking = False

    while True:
        clicking = False

        _grabbed, frame = stream.read()

        if cv2.waitKey(1) == ord("x"):
            break

        hands, image = hand_detector.find_hands(image=frame, draw_box=True, draw_marks=True)

        if len(hands) != 0:
            for hand in hands:
                landmark = hand["landmarks"]

                prediction = model.predict_proba(np.array(landmark).reshape(1, 42))

                if prediction.max() >= min_probability:
                    pred = prediction.argmax()

                    if pred == 0:
                        clicking = True

                        if was_clicking is False:
                            pyautogui.mouseDown()

                    else:
                        clicking = False

                    if pred in [0, 3] and hand["type"] == "Right":
                        if previous_x is None:
                            previous_x = hand["pos_x"]
                            previous_y = hand["pos_y"]

                        x_offset, y_offset = get_mouse_move_offset(hand=hand, previous_x=previous_x, previous_y=previous_y)
                        mouse_tracking_command(x_offset=x_offset, y_offset=y_offset)

                    if pred == 6 and hand["type"] == "Left":
                        pred = 8

                    print(f"Caught the {labels.get(prediction.argmax())} gesture - {i}")

                    i += 1

                    command = commands_table.get(pred, null_command)

                    command(hand_type=hand["type"])

                previous_x = hand["pos_x"]
                previous_y = hand["pos_y"]

        if was_clicking is False and clicking is True:
            was_clicking = True

        if was_clicking is True and clicking is False:
            was_clicking = False
            pyautogui.mouseUp()

    
        cv2.imshow("Video", image)
        
    stream.release()
    cv2.destroyAllWindows()

In [27]:
main()

Starting!!!
Caught the mouse_tracking gesture - 0
Caught the mouse_tracking gesture - 1
Caught the mouse_tracking gesture - 2
Caught the mouse_tracking gesture - 3
Caught the mouse_tracking gesture - 4
Caught the mouse_tracking gesture - 5
Caught the mouse_tracking gesture - 6
Caught the mouse_tracking gesture - 7
Caught the mouse_tracking gesture - 8
Caught the mouse_tracking gesture - 9
Caught the mouse_tracking gesture - 10
Caught the mouse_tracking gesture - 11
Caught the mouse_tracking gesture - 12
Caught the mouse_tracking gesture - 13
Caught the mouse_tracking gesture - 14
Caught the mouse_tracking gesture - 15
Caught the mouse_tracking gesture - 16
Caught the mouse_tracking gesture - 17
Caught the mouse_tracking gesture - 18
Caught the mouse_tracking gesture - 19
Caught the mouse_tracking gesture - 20
Caught the mouse_tracking gesture - 21
Caught the mouse_tracking gesture - 22
Caught the mouse_tracking gesture - 23
Caught the mouse_tracking gesture - 24
Caught the mouse_tracki